# HDF5 in Python

In [283]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import h5py
import tables as tb

In [284]:
np.set_printoptions(precision=2, suppress=True)

In [285]:
# tab-completion for groups and attributes
?h5py.enable_ipython_completer
h5py.enable_ipython_completer()

In [286]:
data_dir = os.path.join(os.getcwd(), 'data')
print(data_dir)

/home/jack/Repos/hdf5-pydata-munich/data


# h5py

The h5py library is a thin, pythonic wrapper around the HDF5 C API.

It tries to expose most of the functionality that the HDF5 library provides.

### Datasets

In [287]:
with h5py.File(name='data/my_h5py_file.h5', mode='w') as f:
    f.create_dataset(name='my_dataset', data=[1.0, 2.7, 3.7, 4.5])
#     f.create_dataset(name='my_other_dataset', data=[1, 2, 3, 4])
#     f.create_dataset(name='my_other_dataset', data=[1, 2, 3, 4], dtype=np.float32)

In [288]:
with h5py.File(name='data/my_h5py_file.h5', mode='r') as f:
    # the array is just a proxy object
    print(f['my_dataset'])
    # the actual data can be accessed with these 2 syntaxes
#     print(f['my_dataset'][:])
#     print(f['my_dataset'][...])

<HDF5 dataset "my_dataset": shape (4,), type "<f8">


### Preallocation on disk

In [289]:
with h5py.File(name='data/my_h5py_file.h5', mode='w') as f:
    ds = f.create_dataset(name='my_dataset', shape=(8, 1))
    ds[0] = 5.2
    ds[1] = 7

### Pick the correct HDF5 datatype

In [290]:
arr = np.array([0, 1, 254, 255, 256, -1, -2], dtype='uint8')
print(arr)

[  0   1 254 255   0 255 254]


In [291]:
with h5py.File(name='data/my_h5py_file.h5', mode='w') as f:
    f.create_dataset(name='my_dataset', shape=(7,), dtype=h5py.h5t.STD_U8BE)
    f['my_dataset'][0:8] = [0, 1, 254, 255, 123456, -1, -2]
    print(f["my_dataset"][:])

[  0   1 254 255 255   0   0]


### Groups

In [292]:
with h5py.File(name='data/my_h5py_file.h5', mode='w') as f:
    f.create_group(name='group1')
    group2 = f.create_group(name='group2')
    group2.create_group(name='group3')

In [293]:
with h5py.File(name='data/my_h5py_file.h5', mode='r') as f:
    group3 = f['group2/group3']
    print(group3.parent)

<HDF5 group "/group2" (1 members)>


### Attributes

In [294]:
with h5py.File(name='data/my_h5py_file.h5', mode='w') as f:
    ds = f.create_dataset(name='my_dataset', data=[1, 2, 3, 4])
    ds.attrs['Unit'] = 'm/s'
    gr = f.create_group(name='my_group')
    gr.attrs['Created'] = '18/12/2017'
    gr.attrs.create(name='Versions', data=np.array([123, 456])) 

### Traverse a HDF5 file with h5py

In [295]:
def print_name(name):
    print(name)

with h5py.File(name='data/my_h5py_file.h5', mode='r') as f:
    f.visit(print_name)

my_dataset
my_group


### HDF5 Command Line Tools

[Here](https://support.hdfgroup.org/products/hdf5_tools/#h5dist) you can find the command line tools developed by the HDF Group. You don't need h5py or PyTables to use them.

If you are on Ubuntu, you can install them with `sudo apt install hdf5-tools`

In [296]:
# -r stands for 'recursive'
!h5ls -r 'data/my_h5py_file.h5'

/                        Group
/my_dataset              Dataset {4}
/my_group                Group


In [297]:
!h5dump 'data/my_h5py_file.h5'

HDF5 "data/my_h5py_file.h5" {
GROUP "/" {
   DATASET "my_dataset" {
      DATATYPE  H5T_STD_I64LE
      DATASPACE  SIMPLE { ( 4 ) / ( 4 ) }
      DATA {
      (0): 1, 2, 3, 4
      }
      ATTRIBUTE "Unit" {
         DATATYPE  H5T_STRING {
            STRSIZE H5T_VARIABLE;
            STRPAD H5T_STR_NULLTERM;
            CSET H5T_CSET_UTF8;
            CTYPE H5T_C_S1;
         }
         DATASPACE  SCALAR
         DATA {
         (0): "m/s"
         }
      }
   }
   GROUP "my_group" {
      ATTRIBUTE "Created" {
         DATATYPE  H5T_STRING {
            STRSIZE H5T_VARIABLE;
            STRPAD H5T_STR_NULLTERM;
            CSET H5T_CSET_UTF8;
            CTYPE H5T_C_S1;
         }
         DATASPACE  SCALAR
         DATA {
         (0): "18/12/2017"
         }
      }
      ATTRIBUTE "Versions" {
         DATATYPE  H5T_STD_I64LE
         DATASPACE  SIMPLE { ( 2 ) / ( 2 ) }
         DATA {
         (0): 123, 456
         }
      }
   }
}
}


---

# PyTables

PyTables provides a higher abstraction over HDF5. This doesn't make it slower than h5py though.

At the moment PyTables does **not** depend on h5py.

### Array


In [298]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_array(where='/', 
                   name='my_array',
#                    name='my-array', # NaturalNameWarning
#                    title='My PyTables Array',
                   obj=[1, 2, 3, 4])

PyTables has a feature called "Natural Naming": nodes (i.e. datasets and groups in the HDF5 file) can be accessed with the dot notation.

In [299]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='r') as f:
    print(f.root.my_array)

/my_array (Array(4,)) ''


### Groups

In [300]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_group(where='/', name='my_group')

### Attributes

In [301]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_array(where=f.root, name='my_array', obj=[1, 2, 3, 4], title='My PyTables Array')
    f.set_node_attr(where='/my_array', attrname='SomeAttribute', attrvalue='SomeValue')
    f.create_group(where='/', name='my_group')
    f.set_node_attr(where='/my_group', attrname='SomeOtherAttribute', attrvalue=123)

### HDF5 datasets have many abstractions in PyTables

Homogenous dataset:

- **Array**
- **CArray**
- **EArray**
- **VLArray**

Heterogenous dataset:

- **Table**

There is also the **Unimplemented** class, to indicate an HDF5 dataset not supported by PyTables.

In [302]:
num_rows = 1000000  # 1 million
num_columns = 5
gaussian = np.random.normal(loc=0, scale=1, size=num_rows).astype('float32')
uniform = np.random.uniform(low=100, high=150, size=num_rows).astype('uint8')
matrix = np.random.random((num_rows, num_columns)).astype('float32')

### Array (again!)

[Docs](http://www.pytables.org/usersguide/libref/homogenous_storage.html#the-array-class)

- Fastest I/O speed
- Homogeneous (i.e. data has same `dtype`)
- Must fit in memory
- Not compressible
- Not enlargeable

In [303]:
%%time
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_array(where='/', name='gaussian', obj=gaussian)
    f.create_array(where='/', name='uniform', obj=uniform)
    f.create_array(where='/', name='matrix', obj=matrix)

CPU times: user 0 ns, sys: 108 ms, total: 108 ms
Wall time: 114 ms


### CArray

[Docs](http://www.pytables.org/usersguide/libref/homogenous_storage.html#carrayclassdescr)

- Chunked storage
- Data must be homogeneous
- Good speed when reading/writing
- Compressible
- Not enlargeable

In [304]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_carray(where='/', name='my_carray', obj=[1, 2, 3, 4])
    # you can create a CArray and fill it later, but you need to specify atom and shape
    carray = f.create_carray(where='/', name='my_other_carray', atom=tb.Float32Atom(), shape=(4, 2))
    # later...
    carray[:, 1] = [5, 6, 7, 8]

In [305]:
filters = tb.Filters(complevel=5, complib='zlib')

Tips on how to use compression (from the PyTables docs)

- A mid-level (5) compression is sufficient. No need to go all the way up (9)
- Use zlib if you must guarantee complete portability
- Use blosc all other times (it is optimized for HDF5)

In [307]:
%%time
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_carray(where='/', name='gaussian', obj=gaussian, filters=filters)
    f.create_carray(where='/', name='uniform', obj=uniform, filters=filters)
    f.create_carray(where='/', name='matrix', obj=matrix, filters=filters)

CPU times: user 960 ms, sys: 80 ms, total: 1.04 s
Wall time: 1.05 s


### EArray

[Docs](http://www.pytables.org/usersguide/libref/homogenous_storage.html#earrayclassdescr)

- Enlargeable on **one** dimension (append)
- Pretty fast at extending, very good at reading
- Data must be homogeneous
- Compressible

In [308]:
%%time
# One (and only one) of the shape dimensions *must* be 0.
# The dimension being 0 means that the resulting EArray object can be extended along it.
# Multiple enlargeable dimensions are not supported right now.
shape = (num_rows, 0)

with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    earray = f.create_earray(where='/',
                             name='my_earray',
                             atom=tb.Float32Atom(),
                             shape=shape,
                             filters=filters)
    earray.append(sequence=matrix[:, 0:1])
    earray.append(sequence=matrix[:, 1:5])

CPU times: user 788 ms, sys: 84 ms, total: 872 ms
Wall time: 877 ms


### VLArray

[Docs](http://www.pytables.org/usersguide/libref/homogenous_storage.html#the-vlarray-class)

- Supports collections of homogeneous data with a variable number of entries
- Compressible
- Enlargeable (append)
- I/O is not very fast
- Like Table datasets, variable length arrays can have only one dimension, and the elements (atoms) of their rows can be fully multidimensional

In [309]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    vlarray = f.create_vlarray(where=f.root, name='my_vlarray', atom=tb.Float32Atom())
    vlarray.append(gaussian[0:10])
    vlarray.append(uniform[0:1000])

### Table

[Docs](http://www.pytables.org/usersguide/libref/structured_storage.html?highlight=table#tableclassdescr)

- Data can be heterogeneous (i.e. different shapes and different dtypes)
- The structure of a table is declared by its description
- It supports *in-kernel* searches with `Table.where`
- It supports multi-column searches
- Non-nested columns can be *indexed*

In order to emulate in Python records mapped to HDF5 C structs PyTables implements a special class so as to easily define all its fields and other properties.

In [310]:
class Particle(tb.IsDescription):
    identity = tb.StringCol(itemsize=22, dflt=' ', pos=0)  # character String
    idnumber = tb.Int16Col(dflt=1, pos=1)  # short integer

In [311]:
print(Particle.columns)

{'identity': StringCol(itemsize=22, shape=(), dflt=b' ', pos=0), 'idnumber': Int16Col(shape=(), dflt=1, pos=1)}


In [312]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    table = f.create_table(where='/', name='my_table', description=Particle)
    
    num_rows = 100
    row = table.row
    for i in range(num_rows):
        row['identity'] = 'I am {}'.format(i)
        row['idnumber'] = i
        row.append()
    # Flush the table buffers to release memory and make sure are written to disk
    table.flush()

A description can be nested inside another description (this will look weird...)

In [313]:
class Particle(tb.IsDescription):
    identity = tb.StringCol(itemsize=22, dflt=' ', pos=0)
    idnumber = tb.Int16Col(dflt=1, pos=1)

    class Properties(tb.IsDescription):
        # 2-D float array (single-precision)
        pressure = tb.Float32Col(shape=(2, 3))
        # 3-D float array (double-precision)
        energy = tb.Float64Col(shape=(2, 3, 4))

In [317]:
with tb.open_file('data/my_pytables_file.h5', 'w') as f:
    table = f.create_table(where='/', name='my_table', description=Particle)
    
    num_rows = 100
    row = table.row
    for i in range(num_rows):
        row['identity'] = 'I am {}'.format(i)
        row['idnumber'] = i
        row['Properties/pressure'] = np.random.random(size=(2, 3))
        row['Properties/energy'] = np.random.random(size=(2, 3, 4))
        row.append()
    table.flush()

### Traverse a HDF5 file with PyTables

In [318]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='r') as f:
    for node in f.walk_nodes('/', classname='Table'):
        print('{}'.format(node._v_pathname))

/my_table


### PyTables utils (CLI)

PyTables is shipped with some useful command line utilities. These CLI utils are in `tables/utils`.

You can use them if you are working in a python environment where you installed PyTables (these CLI utils cause your python interpreter to execute a python script in `tables/scripts`).

In [319]:
# -v stands for 'verbose'
!ptdump -v 'data/my_pytables_file.h5'

/ (RootGroup) ''
/my_table (Table(100,)) ''
  description := {
  "identity": StringCol(itemsize=22, shape=(), dflt=b' ', pos=0),
  "idnumber": Int16Col(shape=(), dflt=1, pos=1),
  "Properties": {
    "energy": Float64Col(shape=(2, 3, 4), dflt=0.0, pos=0),
    "pressure": Float32Col(shape=(2, 3), dflt=0.0, pos=1)}}
  byteorder := 'little'
  chunkshape := (273,)


In [320]:
!pttree --use-si-units --sort-by 'size' 'data/my_pytables_file.h5'


------------------------------------------------------------

/ (RootGroup)
`--my_table (Table)
      mem=24.0kB, disk=65.5kB [100.0%]

------------------------------------------------------------
Total branch leaves:    1
Total branch size:      24.0kB in memory, 65.5kB on disk
Mean compression ratio: 2.73
HDF5 file size:         70.5kB
------------------------------------------------------------



### Let's create some "Big Data"

![Big data, big deal](https://marktortorici.files.wordpress.com/2013/10/ron-burgundy-big-deal.jpg)

In [32]:
%%time

size = int(10e6)
big_data = {
    'uintegers': np.random.randint(0, 255, size, dtype='uint8'),
    'integers': np.random.randint(low=-123, high=456, size=size, dtype='int32'),
    'floats': np.random.normal(loc=0, scale=1, size=size).astype(np.float32),
    'booleans': np.random.choice([True, False], size=size),
}

CPU times: user 972 ms, sys: 296 ms, total: 1.27 s
Wall time: 1.29 s


In [33]:
%%time

class SyntheticDataDescription(tb.IsDescription):
    unsigned_int_field = tb.UInt8Col(pos=0)
    int_field = tb.Int32Col(pos=1)
    float_field = tb.Float32Col(pos=2)
    bool_field = tb.BoolCol(pos=3)


def fill_table(table, data):
    num_records = len(data['integers'])
    print('Fill up the table with {} records'.format(num_records))
    # Get the record object associated with the table:
    row = table.row
    for i in range(num_records):
        row['unsigned_int_field'] = data['uintegers'][i]
        row['int_field'] = data['integers'][i]
        row['float_field'] = data['floats'][i]
        row['bool_field'] = data['booleans'][i]
        row.append()
    # Flush the table buffers
    table.flush()

file_path = os.path.join(data_dir, 'pytables-synthetic-data.h5')
filters = tb.Filters(complevel=5, complib='zlib')

with tb.open_file(file_path, 'w') as f:
    table = f.create_table(
        where='/', name='data_table', description=SyntheticDataDescription,
        title='Synthetic data', filters=filters)

    fill_table(table, big_data)

Fill up the table with 10000000 records
CPU times: user 16.8 s, sys: 536 ms, total: 17.4 s
Wall time: 17.7 s


Querying with numpy is possible only it the table fits in memory. How big is this table?

In [34]:
!pttree --use-si-units 'data/pytables-synthetic-data.h5'


------------------------------------------------------------

/ (RootGroup)
`--data_table (Table)
      mem=100.0MB, disk=65.1MB [100.0%]

------------------------------------------------------------
Total branch leaves:    1
Total branch size:      100.0MB in memory, 65.1MB on disk
Mean compression ratio: 0.65
HDF5 file size:         65.2MB
------------------------------------------------------------



In [35]:
%%time

file_path = os.path.join(data_dir, 'pytables-synthetic-data.h5')

with tb.open_file(file_path, 'r') as f:
    table = f.root.data_table
    results = [x['bool_field'] for x in table.read() 
               if -100 < x['int_field'] < 100 and x['float_field'] > 0.5]
    print('Rows that match the condition: {}'.format(len(results)))

Rows that match the condition: 1060300
CPU times: user 1min 49s, sys: 3.11 s, total: 1min 53s
Wall time: 1min 55s


`table.iterrows()` returns an iterator that iterates over all rows.
This allow us to avoid loading the entire table in memory.

In [36]:
%%time

file_path = os.path.join(data_dir, 'pytables-synthetic-data.h5')

with tb.open_file(file_path, 'r') as f:
    table = f.root.data_table
    results = [x['bool_field'] for x in table.iterrows()
               if -100 < x['int_field'] < 100 and x['float_field'] > 0.5]
    print('Rows that match the condition: {}'.format(len(results)))

Rows that match the condition: 1060300
CPU times: user 2.92 s, sys: 132 ms, total: 3.06 s
Wall time: 3.2 s


`table.where()` uses numepr to make a *in-kernel* query.

In [37]:
%%time

file_path = os.path.join(data_dir, 'pytables-synthetic-data.h5')

with tb.open_file(file_path, 'r') as f:
    table = f.root.data_table
    results = [x['bool_field'] for x in table.where(
            """((-100 < int_field) & (int_field < 100)) & (float_field > 0.5)""")]
    print('Rows that match the condition: {}'.format(len(results)))

Rows that match the condition: 1060300
CPU times: user 812 ms, sys: 72 ms, total: 884 ms
Wall time: 978 ms


Have a look at the huge difference in the CPU time!

[The starving CPU problem (Francesc Alted)](https://python.g-node.org/python-summerschool-2013/_media/starving_cpu/starvingcpus.pdf)

Caveats with the condition in `table.where`!

In [38]:
# This won't work: you can't use Python's standard boolean operators in NumExpr expressions
condition = """(-100 < int_field < 100) & (float_field > 0.5)"""

# This one works
condition = """((-100 < int_field) & (int_field < 100)) & (float_field > 0.5)"""

# Of course you can make it more readable
cond0 = '((-100 < int_field) & (int_field < 100))'
cond1 = '(float_field > 0.5)'
condition = '{} & {}'.format(cond0, cond1)

# NYC Yellow taxi dataset (2015)

This is real world data, and in the CSV files there are some changes from year to year. A few things I noticed:

- in the CSV files from 2014 there is a field called `pickup_datetime`. From January 2015 onwards this field has been renamed as `tpep_pickup_datetime`;
- starting from July 2016 the columns `pickup_longitude` and `pickup_latitude` have been replaced with `PULocationID`, and the columns `dropoff_longitude` and `dropoff_latitude` with `DOLocationID`.

### A quick look at the data

We can have a quick look at the data - without having to load into memory an entire 1.5GB CSV file - with the unix `less` command.

`less yellow_tripdata_2016-01.csv`

### Define a PyTables description

A *description* defines the table structure (basically, the *schema* of your table).

We also define some functions that we will use to populate the table with the data from the CSV files provided by [NYC Taxi and Limousine Commission (TLC)](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml).

In [39]:
class TaxiTableDescription(tb.IsDescription):
    # vendor_id = tb.UInt8Col(pos=0)
    vendor_id = tb.StringCol(8, pos=0)  # 8-character String
    pickup_timestamp_ms = tb.Int64Col(pos=1)
    dropoff_timestamp_ms = tb.Int64Col(pos=2)
    passenger_count = tb.UInt8Col(pos=3)
    trip_distance = tb.Float32Col(pos=4)
    pickup_longitude = tb.Float32Col(pos=5)
    pickup_latitude = tb.Float32Col(pos=6)
    dropoff_longitude = tb.Float32Col(pos=7)
    dropoff_latitude = tb.Float32Col(pos=8)
    fare_amount = tb.Float32Col(pos=9)
    tip_amount = tb.Float32Col(pos=10)
    total_amount = tb.Float32Col(pos=11)

### Create the HDF5 file with an empty table

In [44]:
h5_file_path = os.path.join(data_dir, 'pytables-ny-taxis.h5')
nyc_dir = os.path.join(data_dir, 'nyctaxi')

filters = tb.Filters(complevel=5, complib='zlib')

with tb.open_file(h5_file_path, 'w') as f:
    f.create_table(
        where='/', name='TaxiTable', description=TaxiTableDescription,
        title='NY Yellow Taxi data', filters=filters)

In [40]:
!ptdump -v 'data/pytables-ny-taxis.h5'

/ (RootGroup) ''
/TaxiTable (Table(0,), shuffle, zlib(5)) 'NY Yellow Taxi data'
  description := {
  "vendor_id": UInt8Col(shape=(), dflt=0, pos=0),
  "pickup_timestamp_ms": Int64Col(shape=(), dflt=0, pos=1),
  "dropoff_timestamp_ms": Int64Col(shape=(), dflt=0, pos=2),
  "passenger_count": UInt8Col(shape=(), dflt=0, pos=3),
  "trip_distance": Float32Col(shape=(), dflt=0.0, pos=4),
  "pickup_longitude": Float32Col(shape=(), dflt=0.0, pos=5),
  "pickup_latitude": Float32Col(shape=(), dflt=0.0, pos=6),
  "dropoff_longitude": Float32Col(shape=(), dflt=0.0, pos=7),
  "dropoff_latitude": Float32Col(shape=(), dflt=0.0, pos=8),
  "fare_amount": Float32Col(shape=(), dflt=0.0, pos=9),
  "tip_amount": Float32Col(shape=(), dflt=0.0, pos=10),
  "total_amount": Float32Col(shape=(), dflt=0.0, pos=11)}
  byteorder := 'little'
  chunkshape := (1310,)


Now that we have a HDF5 file, we need to populate the table with the data from the CSV files.

Let's define some functions first.

In [ ]:
def date_to_timestamp_ms(date_obj):
    timestamp_in_nanoseconds = date_obj.astype('int64')
    timestamp_in_ms = (timestamp_in_nanoseconds / 1000000).astype('int64')
    return timestamp_in_ms


def fill_table(table, mapping, df):
    num_records = df.shape[0]  # it's equal to the chunksize used in read_csv
    row = table.row
    for i in range(num_records):
        row['vendor_id'] = df[mapping['vendor_id']].values[i]

        pickup_ms = date_to_timestamp_ms(df[mapping['pickup_datetime']].values[i])
        row['pickup_timestamp_ms'] = pickup_ms
        dropoff_ms = date_to_timestamp_ms(df[mapping['dropoff_datetime']].values[i])
        row['dropoff_timestamp_ms'] = dropoff_ms

        row['passenger_count'] = df['passenger_count'].values[i]
        row['trip_distance'] = df['trip_distance'].values[i]

        row['pickup_longitude'] = df['pickup_longitude'].values[i]
        row['pickup_latitude'] = df['pickup_latitude'].values[i]
        row['dropoff_longitude'] = df['dropoff_longitude'].values[i]
        row['dropoff_latitude'] = df['dropoff_latitude'].values[i]

        row['fare_amount'] = df['fare_amount'].values[i]
        row['tip_amount'] = df['tip_amount'].values[i]
        row['total_amount'] = df['total_amount'].values[i]
        row.append()
    table.flush()


def get_csv_mapping(file_name):
    """CSV files from different year/month have different field names.

    Parameters
    ----------
    fiel_name : str

    Returns
    -------
    mapping : dict

    See Also
    --------
    data dictionary for NY yellow taxi CSV files
    http://www.nyc.gov/html/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf
    """
    # we could also use a regex like '[0-9]){4}-([0-9]){2}\.csv'
    a, b, c = file_name.split('_')
    period, extension = c.split('.')
    year, month = period.split('-')
    # To the left, the key we want to use. To the right, the key in the CSV file
    if year == '2014':
        mapping = {
            'vendor_id': 'vendor_id',
            'pickup_datetime': 'pickup_datetime',
            'dropoff_datetime': 'dropoff_datetime',
        }
    elif year == '2015' or (year == '2016' and 1 <= int(month) <= 6):
        mapping = {
            'vendor_id': 'VendorID',
            'pickup_datetime': 'tpep_pickup_datetime',
            'dropoff_datetime': 'tpep_dropoff_datetime',
        }
    else:
        raise NotImplementedError('data dictionary not defined for this period')

    return mapping

### Populate the table with all CSV files

Processing a single CSV file on my Thinkpad took roughly 20 minutes...

<img src="https://i.imgflip.com/20fb1g.jpg" title="made at imgflip.com"/>

These CSV files are huge! Don't forget the pandas [rule of thumb](http://wesmckinney.com/blog/apache-arrow-pandas-internals/):

> Have 5 to 10 times as much RAM as the size of your dataset.

In [ ]:
# Open the HDF5 file in 'a'ppend mode and populate the table with CSV data
with tb.open_file(h5_file_path, 'a') as f:
    table = f.root.TaxiTable
    # or, in alternative
    # table = list(f.walk_nodes('/', classname='Table'))[0]

    for year in years:
        year_dir = os.path.join(data_dir, 'nyctaxi', year)
        csv_files = os.listdir(year_dir)

        for csv_file in csv_files:
            mapping = get_csv_mapping(csv_file)

            # define the dtype to use when reading the CSV with pandas (this has nothing to do with the HDF5 table)
            dtype = {
                mapping['vendor_id']: 'category',
                'store_and_fwd_flag': 'category',
            }
            parse_dates = [mapping['pickup_datetime'], mapping['dropoff_datetime']]

            t0 = time.time()
            print('Processing {}'.format(csv_file))
            csv_file_path = os.path.join(year_dir, csv_file)

            # read in chunks because these CSV files are too big
            chunksize = 10000
            for chunk in pd.read_csv(
                csv_file_path, chunksize=chunksize, dtype=dtype,
                skipinitialspace=True, parse_dates=parse_dates):
                # print('Processing chunk')
                df = chunk.reset_index(drop=True)
                # for debugging
                # print(df.describe())
                # print(df.head())
                fill_table(table, mapping, df)
                # print('Next chunk')

            t1 = time.time()
            print('Processing {} took {:.2f}s'.format(csv_file, (t1 - t0)))

![He Man I have the data](https://www.storegrowers.com/wp-content/uploads/2016/02/analytics-meme-sword-guy.png)

---

# Reference

- [Introduction to HDF5](https://www.youtube.com/watch?v=BAjsCldRMMc) by Quincey Koziol
- [HDF5 is Eating the World](https://www.youtube.com/watch?v=nddj5OA8LJo) by Andrew Collette
- [HDF5 take 2 - h5py & PyTables](https://www.youtube.com/watch?v=ofLFhQ9yxCw) by Tom Kooij
- [SciPy 2017 notebooks](https://github.com/tomkooij/scipy2017/tree/master/notebooks) by Tom Kooij
- [h5py documentation](http://docs.h5py.org/en/latest/)
- [PyTables documentation](http://www.pytables.org/index.html)